# EcoVoyage Travel — Agent Notebook
This notebook demonstrates a tool-enabled chatbot for a fictional business.

In [ ]:
# Optional: install deps in Colab/Notebook
# !pip install -r business_bot/requirements.txt

In [ ]:
import os, json
from pathlib import Path
from dotenv import load_dotenv
from pdfminer.high_level import extract_text
from openai import OpenAI

from business_bot.tools import record_customer_interest, record_feedback, openai_tools

ROOT = Path('business_bot')
SUMMARY_PATH = ROOT / 'business_summary.txt'
PDF_PATH = ROOT / 'about_business.pdf'

def load_context():
    summary = SUMMARY_PATH.read_text(encoding='utf-8') if SUMMARY_PATH.exists() else ''
    pdf_text = extract_text(str(PDF_PATH)) if PDF_PATH.exists() else ''
    return {'summary': summary, 'pdf': pdf_text}

SYSTEM_PROMPT = ("You are EcoVoyage Travel’s virtual concierge. Use the provided context to answer.\n\n"
                 "If unsure, call record_feedback. Encourage leads to share name/email and call record_customer_interest.")

def build_initial_messages(context):
    return [
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'system', 'content': 'Business summary:
' + context.get('summary','')},
        {'role': 'system', 'content': 'Business PDF:
' + context.get('pdf','')},
    ]

def chat_openai(messages, tools, model='gpt-4o-mini'):
    load_dotenv()
    client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    while True:
        resp = client.chat.completions.create(model=model, messages=messages, tools=tools, tool_choice='auto')
        msg = resp.choices[0].message
        if msg.tool_calls:
            for tc in msg.tool_calls:
                fn = tc.function
                args = json.loads(fn.arguments) if isinstance(fn.arguments, str) else (fn.arguments or {})
                if fn.name == 'record_customer_interest':
                    out = record_customer_interest(args.get('email',''), args.get('name',''), args.get('message',''))
                elif fn.name == 'record_feedback':
                    out = record_feedback(args.get('question',''))
                else:
                    out = 'Unknown tool'
                messages.append({'role': 'tool', 'tool_call_id': tc.id, 'name': fn.name, 'content': out})
            continue
        return msg.content


In [ ]:
# Demo (requires valid OPENAI_API_KEY in .env)
ctx = load_context()
messages = build_initial_messages(ctx)
messages.append({'role':'user','content':'Hi! Do you offer family-friendly trips in Costa Rica?'})
chat_openai(messages, openai_tools)